In [1]:
pip install --upgrade tensorflow==2.15.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!python --version

Python 3.10.10


In [3]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow will run on GPU.")
else:
    print("TensorFlow will run on CPU.")

Num GPUs Available:  1
TensorFlow will run on GPU.


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0


In [5]:
import pandas as pd

# Load the dataset
data_path = '../data/processed/cleaned_diabetes_one_hot_encoding.csv'
df = pd.read_csv(data_path)

# Display the first few rows of the dataframe
print(df.head())

    age  hypertension  heart_disease    bmi  HbA1c_level  blood_glucose_level  \
0  80.0             0              1  25.19          6.6                  140   
1  54.0             0              0  27.32          6.6                   80   
2  28.0             0              0  27.32          5.7                  158   
3  36.0             0              0  23.45          5.0                  155   
4  76.0             1              1  20.14          4.8                  155   

   diabetes  gender_Female  gender_Male  gender_Other  \
0         0              1            0             0   
1         0              1            0             0   
2         0              0            1             0   
3         0              1            0             0   
4         0              0            1             0   

   smoking_history_No Info  smoking_history_current  smoking_history_former  \
0                        0                        0                       0   
1           

In [6]:
# Normalizing the data
from sklearn.preprocessing import MinMaxScaler

features = df.drop('diabetes', axis=1).columns.tolist()
scaler = MinMaxScaler()

df[features] = scaler.fit_transform(df[features])

# Reshape data for LTSM
X = df.drop('diabetes', axis=1).values
Y = df['diabetes'].values

# Reshape X to be [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Splitting the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
# Build the LSTM Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np


model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report, roc_auc_score

# Add precision and recall to the model's metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall()])

# Train the model (ensure you have trained your model with these metrics before evaluating)
history = model.fit(
    X_train, Y_train, 
    epochs=100, 
    batch_size=64, 
    validation_split=0.2, 
    verbose=2
)

# Evaluate the model on the test set with additional metrics
loss, accuracy, precision, recall = model.evaluate(X_test, Y_test, verbose=2)
print(f'Test Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

# For F1-score and ROC-AUC, use sklearn's utilities as they're not directly supported as Keras metrics during training
from sklearn.metrics import f1_score, roc_auc_score

# Predict probabilities for ROC AUC
y_probs = model.predict(X_test)[:, 0]  # assuming your model outputs probabilities for class 1

# Predict class labels for F1 score and other metrics
y_pred = (y_probs > 0.5).astype('int32')

# Calculate F1 score
f1 = f1_score(Y_test, y_pred)
print(f'F1 Score: {f1:.2f}')

# Calculate ROC AUC Score
roc_auc = roc_auc_score(Y_test, y_probs)
print(f'ROC AUC Score: {roc_auc:.2f}')

Epoch 1/100


UnknownError: Graph execution error:

Detected at node CudnnRNN defined at (most recent call last):
<stack traces unavailable>
Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_3334]

##### Refined LTSM Model

In [9]:
# Build the LSTM Model
model = Sequential([
    LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),
    LSTM(25),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Setup callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model
history = model.fit(
    X_train, Y_train, 
    epochs=100, 
    batch_size=64, 
    validation_split=0.2, 
    callbacks=callbacks,
    verbose=2
)

# Load the best model
model.load_weights('best_model.keras')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print(f'Test Accuracy: {accuracy*100:.2f}%')

# Predict class labels for F1 score and other metrics
y_probs = model.predict(X_test)[:, 0]
y_pred = (y_probs > 0.5).astype('int32')

# Calculate F1 score and ROC AUC Score
f1 = f1_score(Y_test, y_pred)
roc_auc = roc_auc_score(Y_test, y_probs)
print(f'F1 Score: {f1:.2f}')
print(f'ROC AUC Score: {roc_auc:.2f}')

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.10909, saving model to best_model.keras
893/893 - 6s - 7ms/step - accuracy: 0.9494 - loss: 0.1641 - val_accuracy: 0.9644 - val_loss: 0.1091
Epoch 2/100

Epoch 2: val_loss improved from 0.10909 to 0.10535, saving model to best_model.keras
893/893 - 3s - 3ms/step - accuracy: 0.9605 - loss: 0.1190 - val_accuracy: 0.9653 - val_loss: 0.1054
Epoch 3/100

Epoch 3: val_loss improved from 0.10535 to 0.10230, saving model to best_model.keras
893/893 - 3s - 3ms/step - accuracy: 0.9617 - loss: 0.1154 - val_accuracy: 0.9662 - val_loss: 0.1023
Epoch 4/100

Epoch 4: val_loss improved from 0.10230 to 0.10195, saving model to best_model.keras
893/893 - 3s - 3ms/step - accuracy: 0.9622 - loss: 0.1135 - val_accuracy: 0.9655 - val_loss: 0.1020
Epoch 5/100

Epoch 5: val_loss improved from 0.10195 to 0.10139, saving model to best_model.keras
893/893 - 3s - 3ms/step - accuracy: 0.9623 - loss: 0.1125 - val_accuracy: 0.9664 - val_loss: 0.1014
Epoch 6/100

E

Epoch 47/100

Epoch 47: val_loss did not improve from 0.07608
893/893 - 3s - 3ms/step - accuracy: 0.9723 - loss: 0.0828 - val_accuracy: 0.9754 - val_loss: 0.0761
Epoch 48/100

Epoch 48: val_loss did not improve from 0.07608
893/893 - 3s - 3ms/step - accuracy: 0.9723 - loss: 0.0827 - val_accuracy: 0.9758 - val_loss: 0.0765
Epoch 49/100

Epoch 49: val_loss improved from 0.07608 to 0.07557, saving model to best_model.keras
893/893 - 3s - 3ms/step - accuracy: 0.9723 - loss: 0.0832 - val_accuracy: 0.9758 - val_loss: 0.0756
Epoch 50/100

Epoch 50: val_loss did not improve from 0.07557
893/893 - 3s - 3ms/step - accuracy: 0.9726 - loss: 0.0819 - val_accuracy: 0.9742 - val_loss: 0.0770
Epoch 51/100

Epoch 51: val_loss did not improve from 0.07557
893/893 - 3s - 3ms/step - accuracy: 0.9729 - loss: 0.0823 - val_accuracy: 0.9760 - val_loss: 0.0757
Epoch 52/100

Epoch 52: val_loss did not improve from 0.07557
893/893 - 3s - 3ms/step - accuracy: 0.9726 - loss: 0.0827 - val_accuracy: 0.9746 - val_los

#### Handling Class Imbalance

In [10]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights for balanced training
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(Y_train), y=Y_train)
class_weight_dict = dict(enumerate(class_weights))

# Retrain model with class weights
history = model.fit(
    X_train, Y_train,
    epochs=100, 
    batch_size=64,
    validation_split=0.2,
    verbose=2,
    class_weight=class_weight_dict
)

Epoch 1/100
893/893 - 4s - 5ms/step - accuracy: 0.9060 - loss: 0.2169 - val_accuracy: 0.8864 - val_loss: 0.1953
Epoch 2/100
893/893 - 3s - 3ms/step - accuracy: 0.9001 - loss: 0.2044 - val_accuracy: 0.8941 - val_loss: 0.1863
Epoch 3/100
893/893 - 3s - 3ms/step - accuracy: 0.9005 - loss: 0.2014 - val_accuracy: 0.9049 - val_loss: 0.1695
Epoch 4/100
893/893 - 3s - 3ms/step - accuracy: 0.9014 - loss: 0.2038 - val_accuracy: 0.9037 - val_loss: 0.1729
Epoch 5/100
893/893 - 3s - 3ms/step - accuracy: 0.9015 - loss: 0.2008 - val_accuracy: 0.8880 - val_loss: 0.2002
Epoch 6/100
893/893 - 3s - 3ms/step - accuracy: 0.8971 - loss: 0.1979 - val_accuracy: 0.8984 - val_loss: 0.1925
Epoch 7/100
893/893 - 3s - 3ms/step - accuracy: 0.9027 - loss: 0.1988 - val_accuracy: 0.8967 - val_loss: 0.1843
Epoch 8/100
893/893 - 3s - 3ms/step - accuracy: 0.8970 - loss: 0.1997 - val_accuracy: 0.8986 - val_loss: 0.1792
Epoch 9/100
893/893 - 3s - 3ms/step - accuracy: 0.8993 - loss: 0.1991 - val_accuracy: 0.8960 - val_loss:

Epoch 74/100
893/893 - 3s - 3ms/step - accuracy: 0.8949 - loss: 0.1885 - val_accuracy: 0.8872 - val_loss: 0.2048
Epoch 75/100
893/893 - 3s - 3ms/step - accuracy: 0.8926 - loss: 0.1915 - val_accuracy: 0.8844 - val_loss: 0.1948
Epoch 76/100
893/893 - 3s - 3ms/step - accuracy: 0.8939 - loss: 0.1905 - val_accuracy: 0.8942 - val_loss: 0.1853
Epoch 77/100
893/893 - 3s - 3ms/step - accuracy: 0.8941 - loss: 0.1893 - val_accuracy: 0.8864 - val_loss: 0.1994
Epoch 78/100
893/893 - 3s - 3ms/step - accuracy: 0.8929 - loss: 0.1918 - val_accuracy: 0.8899 - val_loss: 0.1894
Epoch 79/100
893/893 - 3s - 3ms/step - accuracy: 0.8920 - loss: 0.1918 - val_accuracy: 0.8985 - val_loss: 0.1848
Epoch 80/100
893/893 - 3s - 3ms/step - accuracy: 0.8941 - loss: 0.1903 - val_accuracy: 0.8935 - val_loss: 0.1896
Epoch 81/100
893/893 - 3s - 3ms/step - accuracy: 0.8897 - loss: 0.1928 - val_accuracy: 0.9049 - val_loss: 0.1728
Epoch 82/100
893/893 - 3s - 3ms/step - accuracy: 0.8909 - loss: 0.1916 - val_accuracy: 0.9007 - 

#### Architecture Refinement

In [11]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Dropout layer
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))  # Another Dropout layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall()])

/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# Evaluate the refined model on the test set
loss, accuracy, precision, recall = model.evaluate(X_test, Y_test, verbose=2)
print(f'Refined Model Test Accuracy: {accuracy*100:.2f}%')
print(f'Refined Model Precision: {precision:.2f}')
print(f'Refined Model Recall: {recall:.2f}')

# Calculate and print F1 score and ROC AUC for the refined model
y_probs_refined = model.predict(X_test)[:, 0]  # assuming your model outputs probabilities for class 1
y_pred_refined = (y_probs_refined > 0.5).astype('int32')

# F1 Score
f1_refined = f1_score(Y_test, y_pred_refined)
print(f'Refined Model F1 Score: {f1_refined:.2f}')

# ROC AUC Score
roc_auc_refined = roc_auc_score(Y_test, y_probs_refined)
print(f'Refined Model ROC AUC Score: {roc_auc_refined:.2f}')


558/558 - 1s - 3ms/step - accuracy: 0.4667 - loss: 0.6928 - precision_1: 0.0581 - recall_1: 0.4006
Refined Model Test Accuracy: 46.67%
Refined Model Precision: 0.06
Refined Model Recall: 0.40
558/558 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Refined Model F1 Score: 0.10
Refined Model ROC AUC Score: 0.42


#### Hyperparameter Tuning

In [13]:
pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 985.2 kB/s eta 0:00:00a 0:00:01


In [12]:
pip install --upgrade tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/N/soft/sles15/deeplearning/Python-3.10.10/lib/python3.10/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = parse_requirement(requirement_string)
        File "/N/soft/sles15/deeplearning/Python-3.10.10/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/N/soft/sles15/deeplearning/Python-3.10.10/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/N/soft/

In [11]:
from keras_tuner import RandomSearch
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32)))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='diabetes_prediction'
)

tuner.search(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))
best_model = tuner.get_best_models(num_models=1)[0]

Reloading Tuner from my_dir/diabetes_prediction/tuner0.json

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
480               |256               |units
0.4               |0.1               |dropout

Epoch 1/10


Traceback (most recent call last):
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)


RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/N/u/rajaali/BigRed200/.local/lib/python3.10/site-packages/keras_tuner/src/engine/h

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt

# ROC Curve
y_probs = best_model.predict(X_test)[:, 0]
fpr, tpr, thresholds = roc_curve(Y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(Y_test, y_probs)
pr_auc = auc(recall, precision)

plt.figure()
plt.plot(recall, precision, label=f'PR curve (area = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="upper right")
plt.show()